In [205]:
# !pip install nltk
# !pip install matplotlib
# !pip install tensorflow
# !pip install pandas
# !pip install nltk
# !pip install matplotlib
# !pip install scikit-learn
# !pip install transformers
# !pip install tf-keras
# !pip install datasets

In [206]:
import time
import numpy as np
import pandas as pd
# import seaborn as sns
import string
import re
# sns.set()

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize


import matplotlib.pyplot as plt
import os
import warnings
warnings.filterwarnings("ignore")

# Graphics in retina format are more sharp and legible
%config InlineBackend.figure_format = 'retina'

In [207]:
# use either top model-aware or model-agnostic
# df = pd.read_json('../SHROOM_dev-v2_validation/val.model-agnostic.json')
df = pd.read_json('data/val.model-aware.v2.json')
df["label"] = df["label"].map({"Hallucination":1, "Not Hallucination":0})

df = df.dropna(how="any", axis=1)
df.head()

# also let's think about the tasks, DM, PG, MT

,hyp,ref,src,tgt,model,task,labels,label,p(Hallucination)
0,A sloping top .,tgt,The sides of the casket were covered with heav...,A decorative feature that sits on top of somet...,ltg/flan-t5-definition-en-base,DM,"[Not Hallucination, Hallucination, Not Halluci...",1,0.6
1,To react too much .,tgt,Please try not to overreact if she drives badl...,To react too much or too intensely .,ltg/flan-t5-definition-en-base,DM,"[Not Hallucination, Not Hallucination, Not Hal...",0,0.0
2,The process of spoiling ; the state of being s...,tgt,"To prevent spoilage , store in a cool , dry pl...",The process of spoiling .,ltg/flan-t5-definition-en-base,DM,"[Hallucination, Not Hallucination, Hallucinati...",1,0.6
3,To arrange in a particular way .,tgt,The way the opposition has framed the argument...,To construct in words so as to establish a con...,ltg/flan-t5-definition-en-base,DM,"[Hallucination, Not Hallucination, Not Halluci...",1,0.6
4,A feeling of concern ; a feeling of anxiety .,tgt,To mix with thy concernments i desist . What i...,That in which one is concerned or interested ;...,ltg/flan-t5-definition-en-base,DM,"[Not Hallucination, Hallucination, Hallucinati...",1,0.6


In [208]:
stop_words = stopwords.words('english')
stemmer    = nltk.SnowballStemmer("english")

In [209]:
def clean_text(text):
    '''
        Make text lowercase, remove text in square brackets,remove links,remove punctuation
        and remove words containing numbers.
    '''
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text) # remove urls
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text) # remove punctuation
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

def preprocess_data(text):
    text = clean_text(text)                                                     # Clean puntuation, urls, and so on
    text = ' '.join(word for word in text.split() if word not in stop_words)    # Remove stopwords
#     text = ' '.join(stemmer.stem(word) for word in text.split())                # Stemm all the words in the sentence
    return text

In [210]:
columns = ['hyp','src','tgt'] # exclude task
for x in columns:
    df[x] = df[x].apply(preprocess_data)

# X = np.reshape(X,(499))
# X.shape, y.shape
# X = df.drop(['label','labels', 'model', 'p(Hallucination)','task','ref'], axis=1)
# y = df['label']
df

,hyp,ref,src,tgt,model,task,labels,label,p(Hallucination)
0,sloping top,tgt,sides casket covered heavy black broadcloth ve...,decorative feature sits top something,ltg/flan-t5-definition-en-base,DM,"[Not Hallucination, Hallucination, Not Halluci...",1,0.6
1,react much,tgt,please try overreact drives badly first learni...,react much intensely,ltg/flan-t5-definition-en-base,DM,"[Not Hallucination, Not Hallucination, Not Hal...",0,0.0
2,process spoiling state spoilt,tgt,prevent spoilage store cool dry place meaning ...,process spoiling,ltg/flan-t5-definition-en-base,DM,"[Hallucination, Not Hallucination, Hallucinati...",1,0.6
3,arrange particular way,tgt,way opposition framed argument makes hard us w...,construct words establish context understandin...,ltg/flan-t5-definition-en-base,DM,"[Hallucination, Not Hallucination, Not Halluci...",1,0.6
4,feeling concern feeling anxiety,tgt,mix thy concernments desist meaning concernment,one concerned interested concern affair interest,ltg/flan-t5-definition-en-base,DM,"[Not Hallucination, Hallucination, Hallucinati...",1,0.6
...,...,...,...,...,...,...,...,...,...
496,using gasfired device way stop people using na...,either,doonii fayyadamuun meeshaa geejibuun namootaba...,using ships transport goods far efficient way ...,facebook/nllb-200-distilled-600M,MT,"[Hallucination, Hallucination, Hallucination, ...",1,1.0
497,since montevideo located south equator summer ...,either,وبما أن مونتيفيديو موجودة في جنوب خط الاستواء،...,since montevideo south equator summer winter n...,facebook/nllb-200-distilled-600M,MT,"[Not Hallucination, Not Hallucination, Not Hal...",0,0.0
498,also supporter singapores deputy prime ministe...,either,ginabiabi hiya han deputy prime minister han s...,greeted singapores deputy prime minister wong ...,facebook/nllb-200-distilled-600M,MT,"[Hallucination, Hallucination, Not Hallucinati...",1,0.8
499,concept worm parasitic organisms trained behav...,either,འབུ་ཞེས་པའི་ཐ་སྙད་དེ་ཉིད་འབུ་སྲིན་དཔྱད་རིག་པ་བ...,term bug used entomologists formal sense group...,facebook/nllb-200-distilled-600M,MT,"[Hallucination, Hallucination, Hallucination, ...",1,1.0


In [211]:
# extract preprocessed data as CSV file,
# also i extract them based on tasks, so that they can be fed separately
df = df.drop(['labels'], axis=1)
dm_df = df[df['task'] == 'DM']
mt_df = df[df['task'] == 'MT']
pg_df = df[df['task'] == 'PG']


In [212]:
len(dm_df),len(mt_df), len(pg_df)

(188, 188, 125)

In [213]:
number_of_na = pg_df.isna().sum()
number_of_na

hyp                 0
ref                 0
src                 0
tgt                 0
model               0
task                0
label               0
p(Hallucination)    0
dtype: int64

In [214]:
import pathlib
# file_path = '/Users/byun/Desktop/School/semeval/eda_nlp/thesis_data_before/'
file_path = r"C:\Users\marko\OneDrive\바탕 화면\semeval\Task 6 - windows\eda_nlp-20240601T173748Z-001\thesis_data_before"
file_path_uni = pathlib.Path(file_path).as_posix()
print(file_path_uni)
# Save the DataFrame as a CSV file at the specified location
dm_df.to_csv(file_path_uni + 'dm_df.csv', index=False)
mt_df.to_csv(file_path_uni + 'mt_df.csv', index=False)
pg_df.to_csv(file_path_uni + 'pg_df.csv', index=False)

C:/Users/marko/OneDrive/바탕 화면/semeval/Task 6 - windows/eda_nlp-20240601T173748Z-001/thesis_data_before


In [215]:
# C:/Users/marko/OneDrive/바탕 화면/semeval/Task 6 - windows/eda_nlp-20240601T173748Z-001/thesis_data_after
# task PG, num_aug 16
pg_rd_path = 'C:/Users/marko/OneDrive/바탕 화면/semeval/Task 6 - windows/eda_nlp-20240601T173748Z-001/eda_nlp/thesis_data_after/rd/pg_df_rd.csv'
pg_ri_path = 'C:/Users/marko/OneDrive/바탕 화면/semeval/Task 6 - windows/eda_nlp-20240601T173748Z-001/eda_nlp/thesis_data_after/ri/pg_df_ri.csv'
pg_rs_path = 'C:/Users/marko/OneDrive/바탕 화면/semeval/Task 6 - windows/eda_nlp-20240601T173748Z-001/eda_nlp/thesis_data_after/rs/pg_df_rs.csv'
pg_sr_path = 'C:/Users/marko/OneDrive/바탕 화면/semeval/Task 6 - windows/eda_nlp-20240601T173748Z-001/eda_nlp/thesis_data_after/sr/pg_df_sr.csv'

pg_rd = pd.read_csv(pg_rd_path)
pg_ri = pd.read_csv(pg_ri_path)
pg_rs = pd.read_csv(pg_rs_path)
pg_sr = pd.read_csv(pg_sr_path)

# num_aug 160
path1 = 'C:/Users/marko/OneDrive/바탕 화면/semeval/Task 6 - windows/eda_nlp-20240601T173748Z-001/eda_nlp/thesis_data_after/rd/pg_df_rd.csv'
path2 = 'C:/Users/marko/OneDrive/바탕 화면/semeval/Task 6 - windows/eda_nlp-20240601T173748Z-001/eda_nlp/thesis_data_after/ri/pg_df_ri.csv'
path3 = 'C:/Users/marko/OneDrive/바탕 화면/semeval/Task 6 - windows/eda_nlp-20240601T173748Z-001/eda_nlp/thesis_data_after/rs/pg_df_rs.csv'
path4 = 'C:/Users/marko/OneDrive/바탕 화면/semeval/Task 6 - windows/eda_nlp-20240601T173748Z-001/eda_nlp/thesis_data_after/sr/pg_df_sr.csv'

pg_rd_160 = pd.read_csv(path1)
pg_ri_160 = pd.read_csv(path2)
pg_rs_160 = pd.read_csv(path3)
pg_sr_160 = pd.read_csv(path4)

In [216]:
# task DM 16
path1 = 'C:/Users/marko/OneDrive/바탕 화면/semeval/Task 6 - windows/eda_nlp-20240601T173748Z-001/eda_nlp/thesis_data_after/rd/dm_df_rd.csv'
path2 = 'C:/Users/marko/OneDrive/바탕 화면/semeval/Task 6 - windows/eda_nlp-20240601T173748Z-001/eda_nlp/thesis_data_after/ri/dm_df_ri.csv'
path3 = 'C:/Users/marko/OneDrive/바탕 화면/semeval/Task 6 - windows/eda_nlp-20240601T173748Z-001/eda_nlp/thesis_data_after/rs/dm_df_rs.csv'
path4 = 'C:/Users/marko/OneDrive/바탕 화면/semeval/Task 6 - windows/eda_nlp-20240601T173748Z-001/eda_nlp/thesis_data_after/sr/dm_df_sr.csv'

dm_rd = pd.read_csv(path1)
dm_ri = pd.read_csv(path2)
dm_rs = pd.read_csv(path3)
dm_sr = pd.read_csv(path4)

# 160
path1 = 'C:/Users/marko/OneDrive/바탕 화면/semeval/Task 6 - windows/eda_nlp-20240601T173748Z-001/eda_nlp/thesis_data_after_160/rd/dm_df_rd.csv'
path2 = 'C:/Users/marko/OneDrive/바탕 화면/semeval/Task 6 - windows/eda_nlp-20240601T173748Z-001/eda_nlp/thesis_data_after_160/ri/dm_df_ri.csv'
path3 = 'C:/Users/marko/OneDrive/바탕 화면/semeval/Task 6 - windows/eda_nlp-20240601T173748Z-001/eda_nlp/thesis_data_after_160/rs/dm_df_rs.csv'
path4 = 'C:/Users/marko/OneDrive/바탕 화면/semeval/Task 6 - windows/eda_nlp-20240601T173748Z-001/eda_nlp/thesis_data_after_160/sr/dm_df_sr.csv'

dm_rd_160 = pd.read_csv(path1)
dm_ri_160 = pd.read_csv(path2)
dm_rs_160 = pd.read_csv(path3)
dm_sr_160 = pd.read_csv(path4)

In [217]:
# task MT 16
path1 = 'C:/Users/marko/OneDrive/바탕 화면/semeval/Task 6 - windows/eda_nlp-20240601T173748Z-001/eda_nlp/thesis_data_after/rd/mt_df_rd.csv'
path2 = 'C:/Users/marko/OneDrive/바탕 화면/semeval/Task 6 - windows/eda_nlp-20240601T173748Z-001/eda_nlp/thesis_data_after/ri/mt_df_ri.csv'
path3 = 'C:/Users/marko/OneDrive/바탕 화면/semeval/Task 6 - windows/eda_nlp-20240601T173748Z-001/eda_nlp/thesis_data_after/rs/mt_df_rs.csv'
path4 = 'C:/Users/marko/OneDrive/바탕 화면/semeval/Task 6 - windows/eda_nlp-20240601T173748Z-001/eda_nlp/thesis_data_after/sr/mt_df_sr.csv'

mt_rd = pd.read_csv(path1)
mt_ri = pd.read_csv(path2)
mt_rs = pd.read_csv(path3)
mt_sr = pd.read_csv(path4)

# 160
path1 = 'C:/Users/marko/OneDrive/바탕 화면/semeval/Task 6 - windows/eda_nlp-20240601T173748Z-001/eda_nlp/thesis_data_after_160/rd/mt_df_rd.csv'
path2 = 'C:/Users/marko/OneDrive/바탕 화면/semeval/Task 6 - windows/eda_nlp-20240601T173748Z-001/eda_nlp/thesis_data_after_160/ri/mt_df_ri.csv'
path3 = 'C:/Users/marko/OneDrive/바탕 화면/semeval/Task 6 - windows/eda_nlp-20240601T173748Z-001/eda_nlp/thesis_data_after_160/rs/mt_df_rs.csv'
path4 = 'C:/Users/marko/OneDrive/바탕 화면/semeval/Task 6 - windows/eda_nlp-20240601T173748Z-001/eda_nlp/thesis_data_after_160/sr/mt_df_sr.csv'

mt_rd_160 = pd.read_csv(path1)
mt_ri_160 = pd.read_csv(path2)
mt_rs_160 = pd.read_csv(path3)
mt_sr_160 = pd.read_csv(path4)

In [218]:
#
path1 = 'C:/Users/marko/OneDrive/바탕 화면/semeval/Task 6 - windows/eda_nlp-20240601T173748Z-001/eda_nlp/thesis_data_after/rd/mt_df_rd.csv'
path2 = 'C:/Users/marko/OneDrive/바탕 화면/semeval/Task 6 - windows/eda_nlp-20240601T173748Z-001/eda_nlp/thesis_data_after/ri/mt_df_ri.csv'
path3 = 'C:/Users/marko/OneDrive/바탕 화면/semeval/Task 6 - windows/eda_nlp-20240601T173748Z-001/eda_nlp/thesis_data_after/rs/mt_df_rs.csv'
path4 = 'C:/Users/marko/OneDrive/바탕 화면/semeval/Task 6 - windows/eda_nlp-20240601T173748Z-001/eda_nlp/thesis_data_after/sr/mt_df_sr.csv'

mt_rd = pd.read_csv(path1)
mt_ri = pd.read_csv(path2)
mt_rs = pd.read_csv(path3)
mt_sr = pd.read_csv(path4)

In [219]:
# pg_rd = pg_rd.dropna(how="any", axis=1)
pg_rd = pg_rd.fillna('')
pg_ri = pg_ri.fillna('')
pg_rs = pg_rs.fillna('')
pg_sr = pg_sr.fillna('')

dm_rd = dm_rd.fillna('')
dm_ri = dm_ri.fillna('')
dm_rs = dm_rs.fillna('')
dm_sr = dm_sr.fillna('')

mt_rd = mt_rd.fillna('')
mt_ri = mt_ri.fillna('')
mt_rs = mt_rs.fillna('')
mt_sr = mt_sr.fillna('')

pg_rd_160 = pg_rd_160.fillna('')
pg_ri_160 = pg_ri_160.fillna('')
pg_rs_160 = pg_rs_160.fillna('')
pg_sr_160 = pg_sr_160.fillna('')

dm_rd_160 = dm_rd_160.fillna('')
dm_ri_160 = dm_ri_160.fillna('')
dm_rs_160 = dm_rs_160.fillna('')
dm_sr_160 = dm_sr_160.fillna('')

mt_rd_160 = mt_rd_160.fillna('')
mt_ri_160 = mt_ri_160.fillna('')
mt_rs_160 = mt_rs_160.fillna('')
mt_sr_160 = mt_sr_160.fillna('')
# drop na just deleted the model column i think
mt_rd_160

,hyp,ref,src,tgt,model,label,p(Hallucination),Unnamed: 7
0,john john one,either,,john ann like,,1,1.0,
1,john john one,either,,john ann like,,1,1.0,
2,john john one,either,,john ann like,,1,1.0,
3,john one,either,,john ann,,1,1.0,
4,john john one,either,,john ann like,,1,1.0,
...,...,...,...,...,...,...,...,...
7849,dont money buy dictionary,either,,money buy dictionary,,0,0.0,
7850,dont money buy dictionary,either,,buy dictionary,,0,0.0,
7851,dont money buy dictionary,either,,money buy dictionary,,0,0.0,
7852,dont money buy dictionary,either,,money buy dictionary,,0,0.0,


In [220]:
# mt_rd_160.set_format('tf')

In [221]:
# original dataset before data aug
# X, y = pg_df['hyp'], pg_df['label']
# X, y = dm_df['hyp'], dm_df['label']
# X, y = mt_df['hyp'], mt_df['label']

# task PG
# X, y = pg_rd['hyp'] , pg_rd['label']
# X, y = pg_ri['hyp'] , pg_ri['label']
# X, y = pg_rs['hyp'] , pg_rs['label']
X, y = pg_sr['hyp'] , pg_sr['label']
# task DM
# X, y = dm_rd['hyp'] , dm_rd['label']
# X, y = dm_ri['hyp'] , dm_ri['label']
# X, y = dm_rs['hyp'] , dm_rs['label']
# X, y = dm_sr['hyp'] , dm_sr['label']
# task MT
# X, y = mt_rd['hyp'] , mt_rd['label']
# X, y = mt_ri['hyp'] , mt_ri['label']
# X, y = mt_rs['hyp'] , mt_rs['label']
# X, y = mt_sr['hyp'] , mt_sr['label']

# X, y = df['hyp'], df['label']
# one way of getting label where task is DM, df.loc[df['task'] == 'DM', 'label']
# X, y = df[df['task'] == 'PG']['hyp'], df[df['task'] == 'PG']['label']

# task PG
# X, y = pg_rd_160['hyp'] , pg_rd_160['label']
# X, y = pg_ri_160['hyp'] , pg_ri_160['label']
# X, y = pg_rs_160['hyp'] , pg_rs_160['label']
# X, y = pg_sr_160['hyp'] , pg_sr_160['label']
# task DM
# X, y = dm_rd_160['hyp'] , dm_rd_160['label']
# X, y = dm_ri_160['hyp'] , dm_ri_160['label']
# X, y = dm_rs_160['hyp'] , dm_rs_160['label']
# X, y = dm_sr_160['hyp'] , dm_sr_160['label']
# task MT
# X, y = mt_rd_160['hyp'] , mt_rd_160['label']
# X, y = mt_ri_160['hyp'] , mt_ri_160['label']
# X, y = mt_rs_160['hyp'] , mt_rs_160['label']
# X, y = mt_sr_160['hyp'] , mt_sr_160['label']

In [222]:
print("Class distribution:\n", y.value_counts())

Class distribution:
 label
0    480
1    270
Name: count, dtype: int64


In [223]:
X = np.array(X)

y = np.array(y)


In [224]:
for i in range(len(X)):
    X[i] = str(X[i])

In [225]:
# split into train and test sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,  random_state=42)
len(X_train), len(y_train), len(X_test), len(y_test)

(600, 600, 150, 150)

In [226]:
# count number of none 0 or 1s
number_of_na = df.isna().sum()
print("Number of na:", number_of_na)

Number of na: hyp                 0
ref                 0
src                 0
tgt                 0
model               0
task                0
label               0
p(Hallucination)    0
dtype: int64


In [227]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()

X_train_cv = cv.fit_transform(X_train)
X_test_cv = cv.transform(X_test)

In [228]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf = TfidfTransformer()
X_train_tfidf = tfidf.fit_transform(X_train_cv)

In [229]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb.fit(X_train_cv, y_train)

MultinomialNB()

In [230]:
# predict classes
y_pred = nb.predict(X_test_cv)
y_pred_proba = nb.predict_proba(X_test_cv)[:, 1]

In [231]:
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score

In [232]:
p, r, f, _ = precision_recall_fscore_support(y_test, y_pred,
                                                 average='macro')
print(f'precision is  -> {round(p,2)}')
print(f'recall is -> {round(r,2)}')
print(f'f1_score is -> {round(f,2)}')

precision is  -> 0.98
recall is -> 0.99
f1_score is -> 0.99


In [233]:
# calculate the accuracy
acc = accuracy_score(y_test, y_pred)
print(f'Accuracy is -> {round(acc,2)}')

Accuracy is -> 0.99


In [234]:
cm = confusion_matrix(y_test, y_pred)
print('The confusion_matrix is: \n', cm);

The confusion_matrix is: 
 [[88  2]
 [ 0 60]]


In [235]:
# no import mlxtend? but does not matter too much so skip
# from mlxtend.plotting import plot_confusion_matrix

# fig, ax = plot_confusion_matrix(conf_mat= cm, colorbar=True)
# plt.show()

In [236]:
roc_auc = roc_auc_score(y_test, y_pred_proba)
print(f'The roc_auc Score is -> {round(roc_auc,2)}')

The roc_auc Score is -> 0.99


In [237]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline

pipe = Pipeline([('bow', CountVectorizer()),
                 ('tfidf', TfidfTransformer()),
                 ('model', MultinomialNB())])

pipe.fit(X_train, y_train)

Pipeline(steps=[('bow', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('model', MultinomialNB())])

In [238]:
y_pred = pipe.predict(X_test)

In [239]:
accuracy_score(y_test, y_pred)

0.9333333333333333

In [240]:
confusion_matrix(y_test, y_pred)

array([[85,  5],
       [ 5, 55]], dtype=int64)

In [241]:
# paths to final test data
final_ag_path = 'C:/Users/marko/OneDrive/바탕 화면/semeval/Task 6 - windows/final_test_data/test.model-agnostic.json'
final_aw_path = 'C:/Users/marko/OneDrive/바탕 화면/semeval/Task 6 - windows/final_test_data/test.model-agnostic.json'

final_ag_df = pd.read_json(final_ag_path)
final_aw_df = pd.read_json(final_aw_path)
final_ag_df["label"] = final_ag_df["label"].map({"Hallucination":1, "Not Hallucination":0})
final_aw_df["label"] = final_aw_df["label"].map({"Hallucination":1, "Not Hallucination":0})
# perform preprocessing on final_df AND split it based on tasks!
columns = ['hyp','src','tgt'] # exclude task
for x in columns:
    final_ag_df[x] = final_ag_df[x].apply(preprocess_data)
for x in columns:
    final_aw_df[x] = final_aw_df[x].apply(preprocess_data)
final_ag_pg = final_ag_df[final_ag_df['task'] == 'PG']
final_ag_dm = final_ag_df[final_ag_df['task'] == 'DM']
final_ag_mt = final_ag_df[final_ag_df['task'] == 'MT']

final_aw_pg = final_aw_df[final_aw_df['task'] == 'PG']
final_aw_dm = final_aw_df[final_aw_df['task'] == 'DM']
final_aw_mt = final_aw_df[final_aw_df['task'] == 'MT']

final_ag_df

,id,src,tgt,hyp,task,labels,label,p(Hallucination)
0,1,ты удивишься если я скажу что на самом деле ме...,would surprised told name isnt actually tom,youre gonna surprised say real name isnt tom,MT,"[Not Hallucination, Not Hallucination, Not Hal...",0,0.0
1,2,еды будет полно,plenty food,food full,MT,"[Hallucination, Not Hallucination, Hallucinati...",1,0.8
2,3,думаете том будет меня ждать,think tom wait,think toms gonna wait,MT,"[Not Hallucination, Not Hallucination, Not Hal...",0,0.2
3,6,два брата довольно разные,two brothers pretty different,theres lot friends,MT,"[Hallucination, Hallucination, Hallucination, ...",1,1.0
4,7,infradiaphragmatic intra suprasellar craniopha...,medicine diaphragm,anatomy relating diaphragm,DM,"[Hallucination, Hallucination, Hallucination, ...",1,0.8
...,...,...,...,...,...,...,...,...
1495,2992,я никогда не говорил мэри что чувствую,never told mary felt,ive never told mary feel,MT,"[Hallucination, Not Hallucination, Not Halluci...",0,0.4
1496,2993,beat rat tailed kyoodle runs steers eric laid ...,mutt dog mixed breed little value noisy dog,slang mustang,DM,"[Hallucination, Hallucination, Hallucination, ...",1,1.0
1497,2994,ты знаешь почему они прекратили говорить,know stopped talking,know stopped talking,MT,"[Hallucination, Not Hallucination, Not Halluci...",0,0.4
1498,2996,anyone back,anyone confirm,anyone corroborate,PG,"[Not Hallucination, Not Hallucination, Not Hal...",0,0.0


In [242]:
# import xgboost as xgb

# xgb_pipe = Pipeline([
#                 ('bow', CountVectorizer()),
#                 ('tfidf', TfidfTransformer()),
#                 ('model', xgb.XGBClassifier(
#                     learning_rate=0.1,
#                     max_depth=7,
#                     n_estimators=80,
#                     use_label_encoder=False,
#                     eval_metric='auc'))
#                 ])

In [243]:
# # Fit the pipeline with the data
# xgb_pipe.fit(X_train, y_train)

# y_pred = xgb_pipe.predict(X_test)
# # y_pred_train = pipe.predict(X_train)

# # print('Train: {}'.format(accuracy_score(y_train, y_pred_train)))
# print(f'Accuracy Score For Test Data: -> {round(accuracy_score(y_test, y_pred),2)}')
# print(f'Confusion Matrix Score For Test Data:\n {confusion_matrix(y_test, y_pred)}')
# # calculate the accuracy
# acc = accuracy_score(y_test, y_pred)
# print(f'xgboost Accuracy is -> {round(acc,2)}')

In [244]:
from sklearn.model_selection import cross_val_score

In [245]:
# # Perform cross-validation
# cv_scores = cross_val_score(xgb_pipe, X, y, cv=10)  # 5-fold cross-validation

# # Print the cross-validation scores
# print("Cross-validation scores:", cv_scores)
# print("Mean CV score:", cv_scores.mean())

In [246]:
# final two datasets
ag_X, ag_y = final_ag_pg['hyp'], final_ag_pg['label']
# ag_X, ag_y = final_ag_dm['hyp'], final_ag_dm['label']
# ag_X, ag_y = final_ag_mt['hyp'], final_ag_mt['label']

# ag_X, ag_y = final_aw_pg['hyp'], final_aw_pg['label']
# ag_X, ag_y = final_aw_dm['hyp'], final_aw_dm['label']
# ag_X, ag_y = final_aw_mt['hyp'], final_aw_mt['label']

# ag_X = count_vect.transform(ag_X)
# ag_X = tfidf_transformer.transform(ag_X)

# final_y = xgb_pipe.predict(ag_X)
# final_accuracy_tt = accuracy_score(ag_y, final_y)
# print(f'test/train split xgboost Accuracy is -> {round(acc,2)}')
# print("XGB train/test split prediction with final TEST dataset:", final_accuracy_tt)

In [247]:
# # implement cross validation manually to see if it leads to better accuracy for XGB
# import numpy as np
# import xgboost as xgb
# from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
# from sklearn.metrics import accuracy_score

# # Example data (X should be your feature matrix and y your labels)
# # X, y = ...

# # Number of folds
# n_folds = 5

# # Shuffle and split the data into 5 folds
# indices = np.arange(X.shape[0])
# np.random.shuffle(indices)
# folds = np.array_split(indices, n_folds)

# # Store the accuracy scores for each fold
# accuracy_scores = []
# ag_accuracy_scores = []
# aw_accuracy_scores = []

# # Define the model configuration
# model_config = {
#     'learning_rate': 0.1,
#     'max_depth': 7,
#     'n_estimators': 80,
#     'use_label_encoder': False,
#     'eval_metric': 'auc'
# }
# # model = xgb.XGBClassifier(**model_config)

# for i in range(n_folds):
#     # Prepare the training and validation sets
#     val_indices = folds[i]
#     train_indices = np.hstack([folds[j] for j in range(n_folds) if j != i])

#     X_train, X_val = X[train_indices], X[val_indices]
#     y_train, y_val = y[train_indices], y[val_indices]

#     # Create the pipeline components
#     count_vect = CountVectorizer()
#     tfidf_transformer = TfidfTransformer()

#     # Fit and transform the training data
#     X_train_counts = count_vect.fit_transform(X_train)
#     X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

#     # Fit and transform the validation data
#     X_val_counts = count_vect.transform(X_val)
#     X_val_tfidf = tfidf_transformer.transform(X_val_counts)

#     # Train the model
#     model = xgb.XGBClassifier(**model_config)
#     model.fit(X_train_tfidf, y_train)

#     # Predict on the validation set
#     y_pred = model.predict(X_val_tfidf)

#     # predict using actual final data from semeval (5/17)
# #     ag_X_ag, ag_y_ag = final_ag_df['hyp'], final_ag_df['label']
# #     ag_X_aw, ag_y_aw = final_aw_df['hyp'], final_aw_df['label']
#     # Calculate accuracy
#     accuracy = accuracy_score(y_val, y_pred)
#     accuracy_scores.append(accuracy)

# #     ag_accuracy = accuracy_score()
# #     ag_accuracy_scores.append(ag_accuracy)
# #     aw_accuracy_scores.append(aw_accuracy)

# # Calculate the mean accuracy
# mean_accuracy = np.mean(accuracy_scores)

# print("XGB Cross-validation accuracy scores:", accuracy_scores)
# print("XGB Mean cross-validation accuracy:", mean_accuracy)

# # final two datasets

# ag_X = count_vect.transform(ag_X)
# ag_X = tfidf_transformer.transform(ag_X)
# final_y = model.predict(ag_X)
# final_accuracy = accuracy_score(ag_y, final_y)
# print("XGB final prediction with final TEST dataset:", final_accuracy)

# # train/test split
# print(f'test/train split xgboost Accuracy is -> {round(acc,2)}')
# print("XGB train/test split prediction with final TEST dataset:", final_accuracy_tt)

In [250]:
import numpy as np
from sklearn.model_selection import StratifiedKFold
from transformers import TFBertForSequenceClassification, BertTokenizer, AutoTokenizer, TFAutoModel
import tensorflow as tf
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam

checkpoint = 'bert-base-uncased'
# checkpoint = 'bert-large-uncased'
# Load BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained(checkpoint)
checkpoint = "bert-base-uncased"


In [251]:
import tensorflow as tf
import transformers
print(tf.__version__)
print(transformers.__version__)

2.16.1
4.41.2


In [252]:
# import os
# os.environ['TF_USE_LEGACY_KERAS'] = '1' 

In [253]:
import tf_keras
adam = tf_keras.src.optimizers.Adam(learning_rate=1e-6)

In [254]:
# this here uses manual cross validation!
import numpy as np
import pandas as pd
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import datasets
import time

BATCH_SIZE = 8
EPOCHS = 5
N_SPLITS = 5

# Shuffle and split data into N_SPLITS folds
indices = np.arange(len(X))
np.random.shuffle(indices)
folds = np.array_split(indices, N_SPLITS)

# Initialize lists to store evaluation metrics
all_val_preds = []
accuracies = []
precisions = []
recalls = []
f1s = []

def tokenize_function(example):
    return tokenizer(example['hyp'], truncation=True, padding='max_length', max_length=100)

def order(inp):
    data = list(inp.values())
    return {
        'input_ids': tf.convert_to_tensor(data[1], dtype=tf.int32),
        'token_type_ids': tf.convert_to_tensor(data[2], dtype=tf.int32),
        'attention_mask': tf.convert_to_tensor(data[3], dtype=tf.int32),
    }, data[0]
# changed from optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=1e-5), to without legacy
# bert_model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=1)
# bert_model.compile(
#     # optimizer=Adam,
#     loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
#     metrics=['accuracy']
# )

for i in range(N_SPLITS):
    # Reset model for each fold
    bert_model = TFBertForSequenceClassification.from_pretrained(checkpoint, num_labels=1)
    
    # Compile the model with appropriate loss function and optimizer
    bert_model.compile(
        # first try out lr of 1e-6, then 1e-5, then 1e-4
        # optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=1e-6),
        optimizer=adam,
        loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
        metrics=['accuracy']
    )
    # the code below resets model, and I might try it out? maybe idk
    # Prepare training and validation indices
    val_indices = folds[i]
    train_indices = np.hstack([folds[j] for j in range(N_SPLITS) if j != i])

    # Split the data
    X_train2, X_val = X[train_indices], X[val_indices]
    y_train2, y_val = y[train_indices], y[val_indices]

    # Create Pandas DataFrames
    X_train2_df = pd.DataFrame({'hyp': X_train2, 'label': y_train2})
    X_val_df = pd.DataFrame({'hyp': X_val, 'label': y_val})

    # Convert to HuggingFace Datasets
    df_train = datasets.Dataset.from_pandas(X_train2_df)
    df_val = datasets.Dataset.from_pandas(X_val_df)
    dataset = datasets.DatasetDict({'train': df_train, 'val': df_val})

    # Tokenize the datasets
    tokenized_datasets_mapped = dataset.map(tokenize_function, batched=True, batch_size=BATCH_SIZE, remove_columns=['hyp'])

    # Convert to TensorFlow Datasets
    train_dataset = tf.data.Dataset.from_tensor_slices(tokenized_datasets_mapped['train'][:])
    train_dataset = train_dataset.batch(BATCH_SIZE).shuffle(1000)
    train_dataset = train_dataset.map(order, num_parallel_calls=tf.data.AUTOTUNE)

    val_dataset = tf.data.Dataset.from_tensor_slices(tokenized_datasets_mapped['val'][:])
    val_dataset = val_dataset.batch(BATCH_SIZE)
    val_dataset = val_dataset.map(order, num_parallel_calls=tf.data.AUTOTUNE)

    # Train the model
    start = time.time()
    bert_model.fit(train_dataset, validation_data=val_dataset, epochs=EPOCHS)
    end = time.time()
    elapsed_time = round(end - start, 2)
    print(f'Fitting fold {i+1} is DONE!! Time: {elapsed_time}s')

    # Evaluate the model
    evaluated = bert_model.evaluate(val_dataset)
    val_preds = bert_model.predict(val_dataset).logits
#     val_preds = tf.nn.softmax(val_preds, axis=1).numpy()[:, 1]  # Probability of class 1, 1 try
    # val_preds = tf.nn.sigmoid(val_preds).numpy().squeeze()[:, 1] # 2nd attempt, also i might need try out
    # using a [:, 0] instead?
    # Apply sigmoid activation
    val_preds = tf.nn.sigmoid(val_preds).numpy()

    # Check if val_preds has more than one dimension
    if val_preds.ndim > 1:
        val_preds = val_preds.squeeze()
        # Ensure that there are at least two dimensions to index
        if val_preds.ndim > 1:
            val_preds = val_preds[:, 1]
        else:
            # Handle case where squeezing resulted in a single dimension
            val_preds = val_preds.flatten()
    else:
        val_preds = val_preds.flatten()
    precision, recall, f1, _ = precision_recall_fscore_support(y_val, val_preds.round(), average='weighted')
    acc = accuracy_score(y_val, val_preds.round())

    all_val_preds.append(val_preds)
    accuracies.append(acc)
    precisions.append(precision)
    recalls.append(recall)
    f1s.append(f1)

# Calculate and print average metrics
print(f'Mean Accuracy: {np.mean(accuracies)}')
print(f'Mean Precision: {np.mean(precisions)}')
print(f'Mean Recall: {np.mean(recalls)}')
print(f'Mean F1 Score: {np.mean(f1s)}')


All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Map: 100%|██████████| 150/150 [00:00<00:00, 1309.57 examples/s]


Epoch 1/5
75/75 [==============================] - 330s 4s/step - loss: 0.6599 - accuracy: 0.6550 - val_loss: 0.6693 - val_accuracy: 0.5800
Epoch 2/5
75/75 [==============================] - 333s 4s/step - loss: 0.6444 - accuracy: 0.6550 - val_loss: 0.6609 - val_accuracy: 0.5800
Epoch 3/5
75/75 [==============================] - 357s 5s/step - loss: 0.6392 - accuracy: 0.6550 - val_loss: 0.6515 - val_accuracy: 0.5800
Epoch 4/5
75/75 [==============================] - 361s 5s/step - loss: 0.6220 - accuracy: 0.6550 - val_loss: 0.6394 - val_accuracy: 0.5800
Epoch 5/5
75/75 [==============================] - 341s 5s/step - loss: 0.6109 - accuracy: 0.6550 - val_loss: 0.6193 - val_accuracy: 0.5800
Fitting fold 1 is DONE!! Time: 1723.15s
19/19 [==============================] - 24s 1s/step


All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Map: 100%|██████████| 150/150 [00:00<00:00, 1441.49 examples/s]


Epoch 1/5
75/75 [==============================] - 340s 5s/step - loss: 0.6754 - accuracy: 0.6250 - val_loss: 0.6218 - val_accuracy: 0.7000
Epoch 2/5
75/75 [==============================] - 336s 4s/step - loss: 0.6352 - accuracy: 0.6250 - val_loss: 0.5898 - val_accuracy: 0.7000
Epoch 3/5
75/75 [==============================] - 337s 4s/step - loss: 0.6071 - accuracy: 0.6250 - val_loss: 0.5543 - val_accuracy: 0.7000
Epoch 4/5
75/75 [==============================] - 336s 4s/step - loss: 0.5636 - accuracy: 0.6250 - val_loss: 0.5134 - val_accuracy: 0.7000
Epoch 5/5
75/75 [==============================] - 344s 5s/step - loss: 0.5164 - accuracy: 0.6250 - val_loss: 0.4661 - val_accuracy: 0.7000
Fitting fold 2 is DONE!! Time: 1693.93s
19/19 [==============================] - 25s 1s/step


All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Map: 100%|██████████| 150/150 [00:00<00:00, 1341.13 examples/s]


Epoch 1/5
75/75 [==============================] - 344s 5s/step - loss: 0.6582 - accuracy: 0.6367 - val_loss: 0.6295 - val_accuracy: 0.6533
Epoch 2/5
75/75 [==============================] - 332s 4s/step - loss: 0.6207 - accuracy: 0.6367 - val_loss: 0.5690 - val_accuracy: 0.6533
Epoch 3/5
75/75 [==============================] - 342s 5s/step - loss: 0.5619 - accuracy: 0.6683 - val_loss: 0.4951 - val_accuracy: 0.7267
Epoch 4/5
75/75 [==============================] - 340s 5s/step - loss: 0.4952 - accuracy: 0.7800 - val_loss: 0.4338 - val_accuracy: 0.8400
Epoch 5/5
75/75 [==============================] - 339s 5s/step - loss: 0.4368 - accuracy: 0.8567 - val_loss: 0.3700 - val_accuracy: 0.9067
Fitting fold 3 is DONE!! Time: 1696.4s
19/19 [==============================] - 24s 1s/step


All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Map: 100%|██████████| 150/150 [00:00<00:00, 1388.78 examples/s]


Epoch 1/5
75/75 [==============================] - 329s 4s/step - loss: 0.6483 - accuracy: 0.6400 - val_loss: 0.6402 - val_accuracy: 0.6400
Epoch 2/5
75/75 [==============================] - 327s 4s/step - loss: 0.6131 - accuracy: 0.6400 - val_loss: 0.5838 - val_accuracy: 0.6400
Epoch 3/5
75/75 [==============================] - 326s 4s/step - loss: 0.5608 - accuracy: 0.6417 - val_loss: 0.5230 - val_accuracy: 0.6467
Epoch 4/5
75/75 [==============================] - 326s 4s/step - loss: 0.4785 - accuracy: 0.6883 - val_loss: 0.4310 - val_accuracy: 0.8200
Epoch 5/5
75/75 [==============================] - 325s 4s/step - loss: 0.4061 - accuracy: 0.7900 - val_loss: 0.3864 - val_accuracy: 0.9467
Fitting fold 4 is DONE!! Time: 1632.63s
19/19 [==============================] - 24s 1s/step


All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Map: 100%|██████████| 150/150 [00:00<00:00, 1339.23 examples/s]


Epoch 1/5
75/75 [==============================] - 328s 4s/step - loss: 0.6387 - accuracy: 0.6433 - val_loss: 0.6171 - val_accuracy: 0.6267
Epoch 2/5
75/75 [==============================] - 329s 4s/step - loss: 0.5945 - accuracy: 0.6583 - val_loss: 0.5514 - val_accuracy: 0.7000
Epoch 3/5
75/75 [==============================] - 333s 4s/step - loss: 0.5281 - accuracy: 0.7167 - val_loss: 0.4721 - val_accuracy: 0.7800
Epoch 4/5
75/75 [==============================] - 333s 4s/step - loss: 0.4641 - accuracy: 0.8083 - val_loss: 0.3923 - val_accuracy: 0.8800
Epoch 5/5
75/75 [==============================] - 333s 4s/step - loss: 0.3934 - accuracy: 0.8733 - val_loss: 0.3374 - val_accuracy: 0.9267
Fitting fold 5 is DONE!! Time: 1656.34s
19/19 [==============================] - 24s 1s/step
Mean Accuracy: 0.8426666666666666
Mean Precision: 0.8755966321061408
Mean Recall: 0.8426666666666666
Mean F1 Score: 0.8171055918549854


In [255]:
# Convert to TensorFlow Datasets
def order_final(example):
    return {
        'input_ids': tf.convert_to_tensor(example['input_ids'], dtype=tf.int32),
        'attention_mask': tf.convert_to_tensor(example['attention_mask'], dtype=tf.int32),
        'token_type_ids': tf.convert_to_tensor(example['token_type_ids'], dtype=tf.int32),
    }, tf.convert_to_tensor(example['label'], dtype=tf.int32)


# final two datasets
ag_X, ag_y = final_ag_pg['hyp'], final_ag_pg['label']
# ag_X, ag_y = final_ag_dm['hyp'], final_ag_dm['label']
# ag_X, ag_y = final_ag_mt['hyp'], final_ag_mt['label']

# ag_X, ag_y = final_aw_pg['hyp'], final_aw_pg['label']
# ag_X, ag_y = final_aw_dm['hyp'], final_aw_dm['label']
# ag_X, ag_y = final_aw_mt['hyp'], final_aw_mt['label']

# Create Pandas DataFrames
final_df = pd.DataFrame({'hyp': ag_X, 'label': ag_y})

# Convert to HuggingFace Datasets
final_df2 = datasets.Dataset.from_pandas(final_df)
dataset = datasets.DatasetDict({'final': final_df2})

# Tokenize the datasets
tokenized_datasets_mapped = dataset.map(tokenize_function, batched=True, batch_size=BATCH_SIZE, remove_columns=['hyp'])

# Convert to TensorFlow Datasets
final_dataset = tf.data.Dataset.from_tensor_slices(tokenized_datasets_mapped['final'][:])
final_dataset = final_dataset.batch(BATCH_SIZE)
final_dataset = final_dataset.map(order_final, num_parallel_calls=tf.data.AUTOTUNE)


Map: 100%|██████████| 375/375 [00:00<00:00, 1449.34 examples/s]


In [256]:
# my attempt failed for final dataset
# evaluated = model.evaluate(final_dataset)
final_preds = bert_model.predict(final_dataset).logits
final_preds = tf.nn.sigmoid(final_preds).numpy()
# Check if val_preds has more than one dimension
if final_preds.ndim > 1:
    final_preds = final_preds.squeeze()
    # Ensure that there are at least two dimensions to index
    if final_preds.ndim > 1:
        final_preds = final_preds[:, 1]
    else:
        # Handle case where squeezing resulted in a single dimension
        final_preds = final_preds.flatten()
else:
    final_preds = final_preds.flatten()


47/47 [==============================] - 61s 1s/step


In [257]:
final_preds_binary = [1 if x > 0.5 else 0 for x in final_preds]
# final_preds_mean = np.mean(final_preds, axis=0)
# final_labels = (final_preds_mean >= 0.5).astype(int)
print(f'final aggregate accuracy BERT: {accuracy_score(final_preds_binary, ag_y)}')

# train = pg_rd,final_val = ag_pg: 0.6533333333333333
# train = pg_rd,final_val = ag_dm: 0.517
# train = pg_rd,final_val = ag_mt: 0.537
# jsut using 0s = 0.598

# train = pg_sr, final_val = ag_pg: 0.675
# train = pg_sr, final_val = ag_dm: 0.506
# train = pg_sr, final_val = ag_mt: 0.56

# train = pg_rs, final_val = ag_pg: 0.
# train = pg_rs, final_val = ag_dm: 0.
# train = pg_rs, final_val = ag_mt: 0.

final aggregate accuracy BERT: 0.6426666666666667


In [258]:
final_preds_mean

0.3502739

In [259]:
all_zeros = [0 for x in final_preds]
print(f'all zeros BERT: {accuracy_score(all_zeros, ag_y)}')

all zeros BERT: 0.7413333333333333


In [260]:
# myset = set(final_preds)
# print(myset)
len(ag_y), len(final_preds)

(375, 375)

In [261]:
# i guess right now the big question is, is there a point in using cross validation?
# unless to get best hyper paramter, which i believe i can use some other library for that,
# what is the point? chatgpt said training final model invovles just using only
# train test split so...
final_preds = np.mean(all_val_preds, axis=0)
final_labels = (final_preds >= 0.5).astype(int)

print(f'Accuracy: {np.mean(accuracies):.4f}')
print(f'Precision: {np.mean(precisions):.4f}')
print(f'Recall: {np.mean(recalls):.4f}')
print(f'F1 Score: {np.mean(f1s):.4f}')
print(f'Final Predictions: {final_labels}')

Accuracy: 0.8427
Precision: 0.8756
Recall: 0.8427
F1 Score: 0.8171
Final Predictions: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 1 0 1 0 0 0 1 1 1 0
 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0
 0 1 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 1 0 0 0 0 1 0 0 0
 0 0 1 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 0]


In [262]:
import numpy as np
import pandas as pd
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification
from sklearn.metrics import accuracy_score
import datasets

# Ensure eager execution
tf.config.run_functions_eagerly(True)

BATCH_SIZE = 8
# Create Pandas DataFrame without the index
final_df = pd.DataFrame({'hyp': ag_X, 'label': ag_y})

# Convert to HuggingFace Datasets
final_df2 = datasets.Dataset.from_pandas(final_df)
dataset = datasets.DatasetDict({'final': final_df2})

# Initialize tokenizer (make sure you have the right model name)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize the datasets
def tokenize_function(example):
    return tokenizer(example['hyp'], truncation=True, padding='max_length', max_length=100)

tokenized_datasets_mapped = dataset.map(tokenize_function, batched=True, batch_size=BATCH_SIZE, remove_columns=['hyp'])

# Use from_generator to create dataset with correct output signature
# final_dataset = tf.data.Dataset.from_generator(
#     lambda: iter(tokenized_datasets_mapped['final']),
#     output_signature=(
#         {
#             'input_ids': tf.TensorSpec(shape=(None,), dtype=tf.int32),
#             'attention_mask': tf.TensorSpec(shape=(None,), dtype=tf.int32),
#             'token_type_ids': tf.TensorSpec(shape=(None,), dtype=tf.int32)
#         },
#         tf.TensorSpec(shape=(), dtype=tf.int32)
#     )
# )
final_dataset = tf.data.Dataset.from_tensor_slices(tokenized_datasets_mapped['final'][:])
final_dataset = final_dataset.batch(BATCH_SIZE)
final_dataset = final_dataset.map(order_final, num_parallel_calls=tf.data.AUTOTUNE)

# Evaluate the model
final_preds = bert_model.predict(final_dataset).logits

# Apply sigmoid to get probabilities and flatten the tensor
final_preds = tf.nn.sigmoid(final_preds).numpy().squeeze()

# If final_preds is still multidimensional, reduce it to a 1D array
if final_preds.ndim > 1:
    final_preds = final_preds[:, 0]

# Calculate and print accuracy
accuracy = accuracy_score(ag_y, final_preds.round())
print(f'Final aggregate accuracy BERT: {accuracy}')

# keep track of results here for now
# for task pg, 0.645
# for task dm, 0.453
# for task mt, 0.486

Map: 100%|██████████| 375/375 [00:00<00:00, 1317.01 examples/s]


47/47 [==============================] - 61s 1s/step
Final aggregate accuracy BERT: 0.6426666666666667


In [263]:
# import json
# copy_of_predictions = list(predictions)
# index = 0
# for i in range(len(copy_of_predictions)):
#     copy_of_predictions[i] = list(dict(copy_of_predictions[i]).values())
# #     for key,value in copy_of_predictions[i].items():
# #         copy_of_predictions[i][key] = list(value)

# print(copy_of_predictions)
# with open('mt_sr_160.json','w') as f:
# #     f.write(copy_of_predictions)
#     json.dump(copy_of_predictions, f)

In [264]:
# kf.get_n_splits(X, y)